In [22]:
import torch
import pandas as pd
import json
import os
print(os.getcwd())
# from .stratification.classification.models import PyTorchResNet
from stratification.harness import GEORGEHarness
# from .stratification.harness import GEORGEHarness
# from stratification.utils.utils import set_seed, init_cuda
from stratification.utils.parse_args import get_config
# from stratification.cluster.models.cluster import GaussianMixture
# from stratification.cluster.models.reduction import UMAPReducer

/


In [23]:
# import os

print(os.curdir)

# os.chdir('../')
print(os.curdir)

REPO_DIR = os.getcwd()
with open('configs/celeba_george_config.json', 'r') as f:
    config = json.dumps(json.load(f))
config = get_config([config])

use_cuda_if_available = False  # change to True if you want to use CUDA
use_cuda = use_cuda_if_available and torch.cuda.is_available()
harness = GEORGEHarness(config, use_cuda=use_cuda, log_format='simple')
dataloaders = harness.get_dataloaders(config, mode='erm')
num_classes = dataloaders['train'].dataset.get_num_classes('superclass')
model = harness.get_nn_model(config, num_classes=num_classes, mode='erm')

model.load_state_dict(model = torch.load('/Users/Jen/Documents/UT Austin/Research/hidden-stratification/stratification/checkpoints/celebA/run_2023-11-04_11-48-41_039bb114/erm_2023-11-04_11-49-03_b5f493eb/outputs.pt'))
# model = torch.load('/Users/Jen/Documents/UT Austin/Research/hidden-stratification/stratification/checkpoints/celebA/run_2023-11-04_11-48-41_039bb114/erm_2023-11-04_11-49-03_b5f493eb/outputs.pt')
model.eval()

print(model)


# config = get_config()
# use_cuda = config['use_cuda'] and torch.cuda.is_available()
# set_seed(config['seed'], use_cuda)  # set seeds for reproducibility
# init_cuda(config['deterministic'], config['allow_multigpu'])

# torch.multiprocessing.set_sharing_strategy('file_system')
# harness = GEORGEHarness(config, use_cuda=use_cuda)
# harness.save_full_config(config)

# dataloaders = harness.get_dataloaders(config, mode='erm')
# num_classes = dataloaders['train'].dataset.get_num_classes('superclass')
# model = harness.get_nn_model(config, num_classes=num_classes, mode='erm')


# model = PyTorchResNet()
# model.load_state_dict(state_dict['train']['activations'])

.
.


FileNotFoundError: [Errno 2] No such file or directory: 'configs/celeba_george_config.json'

In [12]:
print(model)

NameError: name 'model' is not defined

In [12]:
print(model.forward())
print(model['test']['activations'].shape)

AttributeError: 'dict' object has no attribute 'forward'

In [ ]:
# df = pd.DataFrame(data, columns=['id', 1, 2, 3, 4, 5]) # n last layers

In [ ]:
attrs_df = pd.read_csv(os.path.join(self.root, 'celebA', 'list_attr_celeba.csv'),
                               delim_whitespace=True)

# Split out filenames and attribute names
self.data_dir = os.path.join(self.root, 'celebA', 'img_align_celeba')
filename_array = attrs_df['image_id'].values
attrs_df = attrs_df.drop(labels='image_id', axis='columns')
attr_names = attrs_df.columns.copy()

# Then cast attributes to numpy array and set them to 0 and 1
# (originally, they're -1 and 1)
attrs_df = attrs_df.values
attrs_df[attrs_df == -1] = 0

def attr_idx(attr_name):
    return attr_names.get_loc(attr_name)

# Get the y values
target_idx = attr_idx(self.target_name)
y_array = attrs_df[:, target_idx]
self._num_supclasses = np.amax(y_array).item() + 1

# Map the confounder attributes to a number 0,...,2^|confounder_idx|-1
self.confounder_idx = [attr_idx(a) for a in self.confounder_names]
confounders = attrs_df[:, self.confounder_idx]
confounder_array = confounders @ np.power(
    2, np.arange(len(self.confounder_idx)))

# Map to groups
self._num_subclasses = self._num_supclasses * \
    pow(2, len(self.confounder_idx))
group_array = (y_array * (self._num_subclasses / 2) +
                confounder_array).astype('int')

# Read in train/val/test splits
split_df = pd.read_csv(os.path.join(self.root, 'celebA', 'list_eval_partition.csv'),
                        delim_whitespace=True)
split_array = split_df['partition'].values
split_dict = {'train': 0, 'val': 1, 'test': 2}
split_indices = split_array == split_dict[self.split]

X = filename_array[split_indices]
Y_dict = {
    'superclass': torch.tensor(y_array[split_indices]),
    'true_subclass': torch.tensor(group_array[split_indices])
}
return X, Y_dict